In [36]:
import pandas as pd
import numpy as np
import re

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier

from sklearn import metrics
data = pd.read_csv("healthy.txt", header=0, \
                    delimiter="\t")

In [37]:
data.head(10)

,﻿CS_ID,Keyword_ID,Keyword,Intent_1,Authority
0,CS-0001,188195,signs of gallbladder problems,Direct Answer,Expert
1,CS-0002,177403,normal blood pressure chart,Resource,General
2,CS-0003,159458,what is normal bloodpressure,Direct Answer,General
3,CS-0004,152734,what is hyperlipidemia,Generic,Expert
4,CS-0005,139086,pill identifier,Generic,General
5,CS-0006,98502,blood clots in leg symptoms,Direct Answer,Expert
6,CS-0007,98115,blood pressure range chart,Resource,General
7,CS-0008,94122,ask webmd question,Navigational,General
8,CS-0009,90914,photos of bed bug bites,Resource,General
9,CS-0010,83756,lyme disease and symptoms,Direct Answer,Expert


In [38]:
data.Intent_1.value_counts()

Generic          43151
Direct Answer    37765
Guide             9138
Resource          7293
Transactional     1238
Navigational      1196
Name: Intent_1, dtype: int64

In [41]:
data.Keyword.describe()

data.Keyword = data.Keyword.str.lower()

In [24]:
#Change User Intent labels to numerical values
data['Intent_1'] = data.Intent_1.map({'Generic': 0 , 'Direct Answer': 1, 'Guide': 2, 'Navigational': 3, 'Resource': 4, 'Transactional': 5 })

#Drop unused columns, null rows, and reindex

health_data = data.drop(['\xef\xbb\xbfCS_ID', 'Keyword_ID', 'Authority'], axis=1)
health_data = health_data.dropna()
health_data = health_data.drop_duplicates('Keyword')
health_data_reindex = health_data.reset_index(drop=True)
health_data_reindex.info()
health_data_reindex.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98140 entries, 0 to 98139
Data columns (total 2 columns):
Keyword     98140 non-null object
Intent_1    98140 non-null float64
dtypes: float64(1), object(1)
memory usage: 2.2+ MB


,Keyword,Intent_1
0,signs of gallbladder problems,1
1,normal blood pressure chart,4
2,what is normal bloodpressure,1
3,what is hyperlipidemia,0
4,pill identifier,0
5,blood clots in leg symptoms,1
6,blood pressure range chart,4
7,ask webmd question,3
8,photos of bed bug bites,4
9,lyme disease and symptoms,1


In [25]:
da_other_intents = healthy[healthy['Intent_1']>1].copy()
da_other_intents.info()
da_other_intents.Intent_1.value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18456 entries, 1 to 99978
Data columns (total 2 columns):
Keyword     18456 non-null object
Intent_1    18456 non-null float64
dtypes: float64(1), object(1)
memory usage: 432.6+ KB


2    9008
4    7064
5    1226
3    1158
Name: Intent_1, dtype: int64

#Function to clean individual keywords
def clean_keywords(keyword):
    stopwords = {'a', 'about', 'above', 'after', 'again', 'against', 'aint', 'all', 'am', 'an', 'and', 'any', 'anybody', 'anyone', 'are', 'arent', 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'between', 'both', 'but', 'by', 'can', 'cant', 'could', 'couldnt', 'couldve', 'd', 'didnt', 'doesnt', 'doing', 'don', 'dont', 'during', 'each', 'early', 'everybody', 'everyone', 'fact', 'few', 'for', 'from', 'further', 'gotta', 'had', 'hadnt', 'has', 'hasnt', 'have', 'havent', 'having', 'he', 'hed', 'her', 'here', 'hers', 'herself', 'hes', 'him', 'himself', 'his', 'i', 'if', 'im', 'in', 'into', 'is', 'isnt', 'it', 'itd', 'itll', 'its', 'itself', 'ive', 'lets', 'll', 'm', 'maam', 'many', 'me', 'mean', 'meaning', 'more', 'most', 'much', 'mustnt', 'mustve', 'my', 'myself', 'need', 'neednt', 'no', 'nobody', 'noone', 'nor', 'not', 'notve', 'now', 'of', 'off', 'on', 'once', 'one', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shall', 'shant', 'she', 'shes', 'should', 'shouldnt', 'shouldve', 'so', 'some', 'somebody', 'someone', 'such', 't', 'than', 'that', 'thats', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'thered', 'therere', 'theres', 'these', 'they', 'theyd', 'theyll', 'theyre', 'theyve', 'this', 'those', 'through', 'too', 'us', 've', 'very', 'was', 'wasnt', 'we', 'were', 'werent', 'weve', 'what', 'whatll', 'whatre', 'whats', 'whatve', 'which', 'while', 'with', 'wont', 'would', 'wouldnt', 'wouldve', 'yet', 'you', 'youd', 'youll', 'your', 'youre', 'yours', 'yourself', 'yourselves', 'youve', 'below', 'best', 'down', 'fun', 'interesting', 'just', 'might', 'mightnt', 'mightve', 'new', 'top', 'under', 'until', 'up', 'will'}
    tokenized_keyword = keyword.split()
    meaningful_words = [kw for kw in tokenized_keyword if not kw in stopwords]
    return( " ".join( meaningful_words ))

da_other_intents['Keyword'] = kws.apply(clean_keywords)
da_other_intents['Keyword'].head(20)

In [26]:
kw_features = da_other_intents.Keyword
kw_target = da_other_intents.Intent_1


X_train, X_test, y_train, y_test = train_test_split(kw_features, kw_target, random_state=1)
print X_train.shape, y_train.shape, X_test.shape, y_test.shape

(13842,) (13842,) (4614,) (4614,)


In [27]:
#Multinomial Naive Bayes
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

text_clf = text_clf.fit(X_train, y_train)
predictions = text_clf.predict(X_test)

print metrics.accuracy_score(y_test, predictions)
print metrics.confusion_matrix(y_test, predictions)

0.939458086367
[[ 200   92    6]
 [   8 1756    3]
 [  10   24  263]]


In [30]:
#SGDClassifier

text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', SGDClassifier(loss='hinge', penalty='l2', n_iter=5, random_state=42)), ])

text_clf_fit = text_clf.fit(X_train, y_train)
predictions = text_clf_fit.predict(X_test)

print metrics.accuracy_score(y_test, predictions)
print metrics.confusion_matrix(y_test, predictions)

cross_val_accuracy = cross_val_score(text_clf, kw_features, kw_target, cv=5).mean()
cross_val_accuracy

0.970307758994
[[2172    2   21    2]
 [  10  268   19    2]
 [  41   17 1741    1]
 [   4    9    9  296]]


0.96353501220959115

In [32]:
cm = metrics.confusion_matrix(y_test,predictions)
cm_df = pd.DataFrame(cm, index=['Predicted GUIDE (2)', 'Predicted NAV (3)', 'Predicted RES (4)', 'Predicted BUY (5)'], 
                     columns=['Actual GUIDE (2)', 'Actual NAV (3)', 'Actual RES (4)', 'Actual BUY (5)'])

cm_df

,Actual GUIDE (2),Actual NAV (3),Actual RES (4),Actual BUY (5)
Predicted GUIDE (2),2172,2,21,2
Predicted NAV (3),10,268,19,2
Predicted RES (4),41,17,1741,1
Predicted BUY (5),4,9,9,296


In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

        2.0       0.98      0.99      0.98      2197
        3.0       0.91      0.90      0.90       299
        4.0       0.97      0.97      0.97      1800
        5.0       0.98      0.93      0.96       318

avg / total       0.97      0.97      0.97      4614



In [35]:
X_test[(y_test == 2) & (predictions == 2)]

7001          best cure for tennis elbow
14260           kidney failure treatment
44215             brain injury treatment
61478      what can i do for acid reflux
35036        treatment for high sed rate
17986             mosquito bite remedies
95833            sun poisoning treatment
98868               intertrigo treatment
19421          treating 2nd degree burns
77547             how to remove fibroids
75519           herbal remedies for copd
16029             treatment for glaucoma
41567         mild sleep apnea treatment
13350          how to use oil of oregano
98032          how to lower hypertension
27343              how to start an essay
18724           treatment for flatulence
95926    how to kill head lice naturally
55373     getting rid of poison ivy rash
79776                 urostomy procedure
46627           how to treat lymph nodes
7366         esophageal reflux treatment
86442              exercise for diabetes
74464          how to incubate duck eggs
48693           

In [45]:
# Output false negatives/positives to file? X_test[(y_test == 0) & (preds == 1)]

In [88]:
#SGDClassifier Gridsearch
from sklearn.grid_search import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2), (1,3)],'tfidf__use_idf': (True, False),'clf__alpha': (1e-2, 1e-3),}

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)

gs_clf = gs_clf.fit(X_train[:400], y_train[:400])

best_parameters, score, _ = max(gs_clf.grid_scores_, key=lambda x: x[1])
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

clf__alpha: 0.01
tfidf__use_idf: True
vect__ngram_range: (1, 1)
